Utilizamos este modelo inicialmente pero por tener una gran cantidad de datos tuvimos que seleccionar un nueo modelo ya que siempre se queda por capacidad, haciendo que se repitiera el proceso varias veces

In [1]:
import pandas as pd
basedatos_df = pd.read_csv('basedatos_modelo_final.csv', sep =';',encoding='utf-8')
basedatos_df.head()

,station_id,cant_unplug_hora,cant_plug_hora,dia,mes,año,dia_semana,hour,Valor_Temp,Valor_Prec,dia_especial
0,1,0,1,1,1,2018,0,4,6.0,0.0,1
1,1,0,1,1,1,2018,0,7,5.0,0.0,1
2,1,2,0,1,1,2018,0,8,5.0,0.0,1
3,1,0,1,1,1,2018,0,9,5.0,0.0,1
4,1,0,8,1,1,2018,0,10,5.0,0.0,1


In [2]:
import pandas as pd
from category_encoders import TargetEncoder
import joblib # Para guardar los encoders

# --- Definir las variables objetivo y las características categóricas ---
target_plug = 'cant_plug_hora'
target_unplug = 'cant_unplug_hora'

# Las columnas categóricas a las que aplicaremos Target Encoding
categorical_cols = ['station_id', 'dia', 'mes', 'hour', 'año', 'dia_semana']

# Features numéricas/binarias (no se les hace Target Encoding)
numerical_or_binary_features = ['Valor_Temp', 'Valor_Prec', 'dia_especial']

# --- Crear y entrenar TargetEncoder para 'cant_plug_hora' ---
print("Entrenando Target Encoder para 'cant_plug_hora'...")
encoder_plug = TargetEncoder(cols=categorical_cols)
encoder_plug.fit(basedatos_df[categorical_cols], basedatos_df[target_plug])
joblib.dump(encoder_plug, 'encoder_plug.pkl')
print("Encoder para 'cant_plug_hora' guardado como 'encoder_plug.pkl'.")


# --- Crear y entrenar TargetEncoder para 'cant_unplug_hora' ---
print("\nEntrenando Target Encoder para 'cant_unplug_hora'...")
encoder_unplug = TargetEncoder(cols=categorical_cols)
encoder_unplug.fit(basedatos_df[categorical_cols], basedatos_df[target_unplug])
joblib.dump(encoder_unplug, 'encoder_unplug.pkl')
print("Encoder para 'cant_unplug_hora' guardado como 'encoder_unplug.pkl'.")


# --- Aplicar los Target Encodings a una copia de los DataFrames para los modelos ---
# Esto creará dos DataFrames separados, cada uno con sus categóricas Target Encoded
# usando el encoder correspondiente a su target.
df_for_model_plug = basedatos_df.copy()
df_for_model_unplug = basedatos_df.copy()

print("\nAplicando Target Encoding a los DataFrames para el entrenamiento...")
df_for_model_plug[categorical_cols] = encoder_plug.transform(df_for_model_plug[categorical_cols])
df_for_model_unplug[categorical_cols] = encoder_unplug.transform(df_for_model_unplug[categorical_cols])

print("Target Encoding aplicado. DataFrames 'df_for_model_plug' y 'df_for_model_unplug' listos.")

# Definir la lista de todas las características para los modelos
# (categóricas ya encodificadas + numéricas/binarias)
base_features_independent = categorical_cols + numerical_or_binary_features

print(f"\nCaracterísticas que se usarán en el modelo: {base_features_independent}")

Entrenando Target Encoder para 'cant_plug_hora'...
Encoder para 'cant_plug_hora' guardado como 'encoder_plug.pkl'.

Entrenando Target Encoder para 'cant_unplug_hora'...
Encoder para 'cant_unplug_hora' guardado como 'encoder_unplug.pkl'.

Aplicando Target Encoding a los DataFrames para el entrenamiento...
Target Encoding aplicado. DataFrames 'df_for_model_plug' y 'df_for_model_unplug' listos.

Características que se usarán en el modelo: ['station_id', 'dia', 'mes', 'hour', 'año', 'dia_semana', 'Valor_Temp', 'Valor_Prec', 'dia_especial']


In [3]:
from sklearn.model_selection import train_test_split
import pandas as pd # Asegúrate de tener pandas importado aquí si ejecutas solo este bloque

# Las variables 'df_for_model_plug', 'df_for_model_unplug', 'target_plug', 'target_unplug',
# y 'base_features_independent' deben estar definidas desde el Paso 1.

print("\n--- División de datos para el Modelo de Enganches (cant_plug_hora) ---")
X1_features_for_model = base_features_independent
X1 = df_for_model_plug[X1_features_for_model]
y1 = df_for_model_plug[target_plug]

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=42)

print(f"Dimensiones X1_train (plug): {X1_train.shape}")
print(f"Dimensiones X1_test (plug): {X1_test.shape}")
print(f"Dimensiones y1_train (plug): {y1_train.shape}")
print(f"Dimensiones y1_test (plug): {y1_test.shape}")

print("\n--- División de datos para el Modelo de Desenganches (cant_unplug_hora) ---")
X2_features_for_model = base_features_independent
X2 = df_for_model_unplug[X2_features_for_model]
y2 = df_for_model_unplug[target_unplug]

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

print(f"Dimensiones X2_train (unplug): {X2_train.shape}")
print(f"Dimensiones X2_test (unplug): {X2_test.shape}")
print(f"Dimensiones y2_train (unplug): {y2_train.shape}")
print(f"Dimensiones y2_test (unplug): {y2_test.shape}")

print("\nDatos divididos en conjuntos de entrenamiento y prueba.")


--- División de datos para el Modelo de Enganches (cant_plug_hora) ---
Dimensiones X1_train (plug): (5585712, 9)
Dimensiones X1_test (plug): (1396429, 9)
Dimensiones y1_train (plug): (5585712,)
Dimensiones y1_test (plug): (1396429,)

--- División de datos para el Modelo de Desenganches (cant_unplug_hora) ---
Dimensiones X2_train (unplug): (5585712, 9)
Dimensiones X2_test (unplug): (1396429, 9)
Dimensiones y2_train (unplug): (5585712,)
Dimensiones y2_test (unplug): (1396429,)

Datos divididos en conjuntos de entrenamiento y prueba.


In [4]:
from sklearn.ensemble import RandomForestRegressor # Importa Random Forest
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import joblib
import pandas as pd # Asegúrate de tener pandas importado aquí si ejecutas solo este bloque


# Las variables X1_train, X1_test, y1_train, y1_test, X2_train, X2_test, y2_train, y2_test
# deben estar definidas desde el Paso 2.

print("\n--- Entrenamiento y Evaluación para el MODELO PLUG (cant_plug_hora) ---")

# --- Modelo Random Forest para Enganches ---
# Se recomienda un n_estimators más bajo que LightGBM para empezar,
# ya que Random Forest es más intensivo computacionalmente por árbol.
# Puedes ajustar n_estimators y otros parámetros como max_depth, min_samples_leaf.
model_plug_rf = RandomForestRegressor(n_estimators=30, random_state=42, n_jobs=-1) # n_jobs=-1 usa todos los cores
print("Entrenando Random Forest para cant_plug_hora...")
model_plug_rf.fit(X1_train, y1_train)

predictions_plug_rf = model_plug_rf.predict(X1_test)
mae_plug_rf = mean_absolute_error(y1_test, predictions_plug_rf)
mse_plug_rf = mean_squared_error(y1_test, predictions_plug_rf)
rmse_plug_rf = np.sqrt(mse_plug_rf)
r2_plug_rf = r2_score(y1_test, predictions_plug_rf)

print(f'\nMétricas Random Forest para cant_plug_hora:')
print(f'  MAE: {mae_plug_rf:.2f}')
print(f'  MSE: {mse_plug_rf:.2f}')
print(f'  RMSE: {rmse_plug_rf:.2f}')
print(f'  R²: {r2_plug_rf:.2f}')
#joblib.dump(model_plug_rf, 'model_plug_rf.pkl')
print("Modelo 'model_plug_rf.pkl' guardado.")





--- Entrenamiento y Evaluación para el MODELO PLUG (cant_plug_hora) ---
Entrenando Random Forest para cant_plug_hora...

Métricas Random Forest para cant_plug_hora:
  MAE: 1.43
  MSE: 4.18
  RMSE: 2.04
  R²: 0.47
Modelo 'model_plug_rf.pkl' guardado.


In [5]:
print("\n--- Entrenamiento y Evaluación para el MODELO UNPLUG (cant_unplug_hora) ---")

# --- Modelo Random Forest para Desenganches ---
model_unplug_rf = RandomForestRegressor(n_estimators=30, random_state=42, n_jobs=-1)
print("Entrenando Random Forest para cant_unplug_hora...")
model_unplug_rf.fit(X2_train, y2_train)

predictions_unplug_rf = model_unplug_rf.predict(X2_test)
mae_unplug_rf = mean_absolute_error(y2_test, predictions_unplug_rf)
mse_unplug_rf = mean_squared_error(y2_test, predictions_unplug_rf)
rmse_unplug_rf = np.sqrt(mse_unplug_rf)
r2_unplug_rf = r2_score(y2_test, predictions_unplug_rf)

print(f'\nMétricas Random Forest para cant_unplug_hora:')
print(f'  MAE: {mae_unplug_rf:.2f}')
print(f'  MSE: {mse_unplug_rf:.2f}')
print(f'  RMSE: {rmse_unplug_rf:.2f}')
print(f'  R²: {r2_unplug_rf:.2f}')
#joblib.dump(model_unplug_rf, 'model_unplug_rf.pkl')
print("Modelo 'model_unplug_rf.pkl' guardado.")


--- Entrenamiento y Evaluación para el MODELO UNPLUG (cant_unplug_hora) ---
Entrenando Random Forest para cant_unplug_hora...


MemoryError: could not allocate 536870912 bytes

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import pandas as pd # Asegúrate de importar pandas

# Ejemplo de muestreo del 10%
# Si tus datos son realmente muy grandes, puedes empezar con 0.01 (1%) o 0.05 (5%)
sample_fraction = 0.05
subset_size = int(X1_train.shape[0] * sample_fraction)

# Genera índices aleatorios para el subset
np.random.seed(42) # Para reproducibilidad
random_indices = np.random.choice(X1_train.index, size=subset_size, replace=False)

X_train_subset = X1_train.loc[random_indices]
y_train_subset = y1_train.loc[random_indices]

print(f"Tamaño del subset de entrenamiento para tuning: {X_train_subset.shape}")
print(f"Tamaño de las etiquetas del subset: {y_train_subset.shape}")


# --- 2. Reducir la cuadrícula de hiperparámetros (param_grid) ---
# Usa rangos más pequeños y menos opciones, especialmente para n_estimators.
param_grid = {
    'n_estimators': [50, 60],  # ¡Valores MUCHO más pequeños para empezar!
    'max_depth': [5, 10, 15],     # Profundidades más limitadas
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 3]
}


# --- 3. Inicializar el modelo Random Forest Regressor ---
rf_model = RandomForestRegressor(random_state=42)

# --- 4. Inicializar GridSearchCV ---
# n_jobs=-1 usa todos los núcleos de tu CPU para acelerar (si tienes memoria suficiente por núcleo)
# cv=3 usa 3-fold cross-validation, reduce el tiempo de evaluación
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid,
                           cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# --- 5. Entrenar GridSearchCV en el SUBSET ---
print("\nIniciando GridSearchCV en el subset de datos...")
grid_search.fit(X_train_subset, y_train_subset)

# --- 6. Mostrar los mejores parámetros y el mejor score ---
print("\nMejores parámetros encontrados:", grid_search.best_params_)
print("Mejor score (MSE negativo):", grid_search.best_score_)

# El mejor modelo entrenado lo obtienes así:
best_rf_model = grid_search.best_estimator_
print("\nMejor modelo obtenido de GridSearchCV:", best_rf_model)


# Opcional: Entrenar el mejor modelo en el *conjunto de entrenamiento COMPLETO*
# print("\nEntrenando el mejor modelo con el conjunto de entrenamiento COMPLETO...")
# best_rf_model_full = RandomForestRegressor(**grid_search.best_params_, random_state=42)
# best_rf_model_full.fit(X1_train_encoded, y1_train)
# print("Modelo entrenado en conjunto completo.")

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import pandas as pd # Asegúrate de importar pandas

# Ejemplo de muestreo del 10%
# Si tus datos son realmente muy grandes, puedes empezar con 0.01 (1%) o 0.05 (5%)
sample_fraction = 0.05
subset_size = int(X1_train.shape[0] * sample_fraction)

# Genera índices aleatorios para el subset
np.random.seed(42) # Para reproducibilidad
random_indices = np.random.choice(X1_train.index, size=subset_size, replace=False)

X_train_subset = X1_train.loc[random_indices]
y_train_subset = y1_train.loc[random_indices]

print(f"Tamaño del subset de entrenamiento para tuning: {X_train_subset.shape}")
print(f"Tamaño de las etiquetas del subset: {y_train_subset.shape}")


# --- 2. Reducir la cuadrícula de hiperparámetros (param_grid) ---
# Usa rangos más pequeños y menos opciones, especialmente para n_estimators.
param_grid = {
    'n_estimators': [50, 60],  # ¡Valores MUCHO más pequeños para empezar!
    'max_depth': [5, 10, 15],     # Profundidades más limitadas
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 3]
}


# --- 3. Inicializar el modelo Random Forest Regressor ---
rf_model = RandomForestRegressor(random_state=42)

# --- 4. Inicializar GridSearchCV ---
# n_jobs=-1 usa todos los núcleos de tu CPU para acelerar (si tienes memoria suficiente por núcleo)
# cv=3 usa 3-fold cross-validation, reduce el tiempo de evaluación
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid,
                           cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# --- 5. Entrenar GridSearchCV en el SUBSET ---
print("\nIniciando GridSearchCV en el subset de datos...")
grid_search.fit(X_train_subset, y_train_subset)

# --- 6. Mostrar los mejores parámetros y el mejor score ---
print("\nMejores parámetros encontrados:", grid_search.best_params_)
print("Mejor score (MSE negativo):", grid_search.best_score_)

# El mejor modelo entrenado lo obtienes así:
best_rf_model = grid_search.best_estimator_
print("\nMejor modelo obtenido de GridSearchCV:", best_rf_model)


# Opcional: Entrenar el mejor modelo en el *conjunto de entrenamiento COMPLETO*
# print("\nEntrenando el mejor modelo con el conjunto de entrenamiento COMPLETO...")
# best_rf_model_full = RandomForestRegressor(**grid_search.best_params_, random_state=42)
# best_rf_model_full.fit(X1_train_encoded, y1_train)
# print("Modelo entrenado en conjunto completo.")

Tamaño del subset de entrenamiento para tuning: (232650, 8)
Tamaño de las etiquetas del subset: (232650,)

Iniciando GridSearchCV en el subset de datos...
Fitting 3 folds for each of 24 candidates, totalling 72 fits

Mejores parámetros encontrados: {'max_depth': 15, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 60}
Mejor score (MSE negativo): -4.511792169993154

Mejor modelo obtenido de GridSearchCV: RandomForestRegressor(max_depth=15, min_samples_leaf=3, n_estimators=60,
                      random_state=42)
